In [1]:
%run C:/Users/MohammedSB/Desktop/projects/Hypertension/Requirements.ipynb

In [2]:
# Seed
set_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
PATH = r"D:\\MohammedSB\\Fundus\\"
CSV_PATH = {"HTNPath": PATH + r"HTN", "NonHTNPath": PATH + "NonHTN"}

MODELS_PATH = r"C:\Users\MohammedSB\Desktop\projects\Hypertension\models"

In [5]:
train_set, test_set = get_datasets()

In [6]:
train_transform = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
    T.RandomHorizontalFlip(0.5),
    T.RandomRotation(degrees=(0, 360)),
    T.GaussianBlur(5),
    T.Normalize(mean=[0.3675, 0.1760, 0.1047], std=[0.2289, 0.1180, 0.0748]),
])

test_transform = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
    T.Normalize(mean=[0.3675, 0.1760, 0.1047], std=[0.2289, 0.1180, 0.0748]),
])

train_dataset = HypertensionDataset(CSV_PATH, train_set, train_transform=train_transform)
test_dataset = HypertensionDataset(CSV_PATH, test_set, test_transform=test_transform)

train_loader, test_loader = train_test_loader(train_dataset, test_dataset, batch_size=8)

# val_dataset = HypertensionDataset(CSV_PATH, val_set, test_transform=test_transform)
# val_loader = DataLoader(val_dataset, batch_size=8)

In [7]:
FM_PATH = MODELS_PATH + r"\FundusModel.pth"
DM_PATH = MODELS_PATH + r"\DemographicFCNN.pth"

criterion = nn.BCEWithLogitsLoss()

# CombineFeatures (Joint Fusion)

In [8]:
tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
)

image_model = get_densenet201(device=device, freeze=True, with_mlp=False, outputs=32)

fusion_model = nn.Sequential(
    nn.Linear(in_features=64, out_features=128),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

tabular_model, image_model, fusion_model = tabular_model.to(device).float(), image_model.to(device).float(),\
                                                fusion_model.to(device).float()

In [9]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

early_stop = {
    "patience": 5,
    "min_delta": 0.001,
    "multip": 5,
}

optimizer = torch.optim.Adam([
    {'params': image_model.parameters(), 'lr': 1e-4},
    {'params': tabular_model.parameters(), 'lr': 1e-4},
    {'params': fusion_model.parameters(), 'lr': 1e-4}],
)

In [10]:
epochs = 200
metrics = train_val(epochs=epochs, models=models, criterion=criterion, optimizer=optimizer, train_loader=train_loader,
                    val_loader=test_loader, device=device, early_stop=early_stop)

Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

Beginning Training: 

Epoch 1/200


Batch in Progress:   0%|                                                     | 0/29 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.702244,    Accuracy: 48.52%,    Correct Counter: 444/915,    F1 Score: 0.65,    Precision: 0.49,    Recall:  1.00 



Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

...

Validation Set:
    Average Loss:  0.604115,    Accuracy: 66.67%,    Correct Counter: 152/228,    F1 Score: 0.73,    Precision: 0.61,    Recall:  0.90 

Epoch 25/200


Batch in Progress:   0%|                                                     | 0/29 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.556352,    Accuracy: 71.91%,    Correct Counter: 658/915,    F1 Score: 0.76,    Precision: 0.65,    Recall:  0.92 



Batch in Progress: 100%|████████████████████████████████████████████| 29/29 [00:07<00:00,  3.97it/s]

Validation Set:
    Average Loss:  0.603954,    Accuracy: 67.11%,    Correct Counter: 153/228,    F1 Score: 0.73,    Precision: 0.61,    Recall:  0.90 



In [11]:
torch.save(image_model.state_dict(), MODELS_PATH+"\CombineFeatures\FundusPath.pth")
torch.save(tabular_model.state_dict(), MODELS_PATH+"\CombineFeatures\DemographicPath.pth")
torch.save(fusion_model.state_dict(), MODELS_PATH+"\CombineFeatures\FusionPath.pth")

# CombineOutputs (Joint Fusion)

In [12]:
tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

image_model = get_densenet201(device=device, freeze=True, with_mlp=True, outputs=1)

fusion_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=4),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=4, out_features=1),
)

tabular_model, image_model, fusion_model = tabular_model.to(device).float(), image_model.to(device).float(),\
                                                fusion_model.to(device).float()

In [13]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

optimizer = torch.optim.Adam([
    {'params': image_model.parameters(), 'lr': 1e-4},
    {'params': tabular_model.parameters(), 'lr': 1e-4},
    {'params': fusion_model.parameters(), 'lr': 1e-4}],
)

epochs = 200
metrics = train_val(epochs=epochs, models=models, criterion=criterion, optimizer=optimizer, train_loader=train_loader,
                    val_loader=test_loader, device=device, early_stop=early_stop)

Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

Beginning Training: 

Epoch 1/200


Batch in Progress:   0%|                                                     | 0/29 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.702929,    Accuracy: 48.52%,    Correct Counter: 444/915,    F1 Score: 0.65,    Precision: 0.49,    Recall:  1.00 



Batch in Progress:   0%|                                                    | 0/115 [00:00<?, ?it/s]

...

Validation Set:
    Average Loss:  0.607914,    Accuracy: 69.30%,    Correct Counter: 158/228,    F1 Score: 0.73,    Precision: 0.64,    Recall:  0.86 

Epoch 130/200


Batch in Progress:   0%|                                                     | 0/29 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.599915,    Accuracy: 73.11%,    Correct Counter: 669/915,    F1 Score: 0.76,    Precision: 0.68,    Recall:  0.86 



Batch in Progress: 100%|████████████████████████████████████████████| 29/29 [00:07<00:00,  3.95it/s]

Validation Set:
    Average Loss:  0.605971,    Accuracy: 69.30%,    Correct Counter: 158/228,    F1 Score: 0.73,    Precision: 0.65,    Recall:  0.84 



In [14]:
torch.save(image_model.state_dict(), MODELS_PATH + "\CombineOutputs\FundusPath.pth")
torch.save(tabular_model.state_dict(), MODELS_PATH + "\CombineOutputs\DemographicPath.pth")
torch.save(fusion_model.state_dict(), MODELS_PATH + "\CombineOutputs\FusionPath.pth")

## KeepFeatures (Late Fusion)

In [15]:
image_model = get_densenet201(device=device, freeze=True, with_mlp=True, outputs=1)
image_model.load_state_dict(torch.load(FM_PATH))

image_model = image_model.to(device).float()

In [16]:
models = {
    "image_model": image_model,
    "tabular_model": None,
    "fusion_model": None,
}

In [17]:
train_x, train_y = build_tabular_dataset(models, train_dataset, method="keep_features")
test_x, test_y = build_tabular_dataset(models, test_dataset, method="keep_features")

train_fusion_set = InputOutputDataset(train_x, train_y)
test_fusion_set = InputOutputDataset(test_x, test_y)

train_fusion_loader = DataLoader(train_fusion_set, batch_size=32)
test_fusion_loader = DataLoader(test_fusion_set, batch_size=32)

## XGBoost

In [18]:
boost = xgb.XGBClassifier(tree_method='gpu_hist', objective="binary:logistic")

In [19]:
param_grid = {'max_depth': [3,6,10],
           'learning_rate': [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.3],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

clf = GridSearchCV(estimator=boost, 
                   param_grid=param_grid,
                   scoring='f1', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed: 14.3min finished


0.7661085136572554


In [20]:
with open(MODELS_PATH+'/KeepFeatures/XGBParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

## SVM

In [21]:
svm = SVC()

In [22]:
param_grid = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']},
    {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'degree': [1,2,3], 'kernel': ['poly']}
]

clf = GridSearchCV(estimator=svm, 
                   param_grid=param_grid,
                   scoring='f1', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

Fitting 5 folds for each of 68 candidates, totalling 340 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.7631837442734091


[Parallel(n_jobs=1)]: Done 340 out of 340 | elapsed:   12.4s finished


In [23]:
with open(MODELS_PATH+'/KeepFeatures/SVMParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

## FCNN

In [24]:
fusion_model = nn.Sequential(
    nn.Linear(in_features=3, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

fusion_model = fusion_model.to(device).float()

In [25]:
models = {
    "image_model": None,
    "tabular_model": None,
    "fusion_model": fusion_model,
}

early_stop = {
    "patience": 20,
    "min_delta": 0,
    "multip": 10
}

optimizer = torch.optim.Adam([
    {'params': fusion_model.parameters(), 'lr': 1e-4}],
)

In [26]:
epochs = 1000
metrics = train_val(epochs=epochs, models=models, criterion=criterion, optimizer=optimizer, train_loader=train_fusion_loader,
                    val_loader=test_fusion_loader, device=device, early_stop=early_stop)

Batch in Progress:  66%|████████████████████████████▏              | 19/29 [00:00<00:00, 170.72it/s]

Beginning Training: 

Epoch 1/1000


Batch in Progress:   0%|                                                      | 0/8 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.693390,    Accuracy: 42.95%,    Correct Counter: 393/915,    F1 Score: 0.57,    Precision: 0.45,    Recall:  0.78 



Batch in Progress: 100%|███████████████████████████████████████████| 29/29 [00:00<00:00, 200.81it/s]

...

Train Set:
    Average Loss:  0.541123,    Accuracy: 71.04%,    Correct Counter: 650/915,    F1 Score: 0.75,    Precision: 0.65,    Recall:  0.88 

Validation Set:
    Average Loss:  0.589746,    Accuracy: 68.42%,    Correct Counter: 156/228,    F1 Score: 0.74,    Precision: 0.63,    Recall:  0.88 

Epoch 67/1000
Train Set:
    Average Loss:  0.540984,    Accuracy: 71.04%,    Correct Counter: 650/915,    F1 Score: 0.75,    Precision: 0.65,    Recall:  0.88 

Validation Set:
    Average Loss:  0.589961,    Accuracy: 68.42%,    Correct Counter: 156/228,    F1 Score: 0.74,    Precision: 0.63,    Recall:  0.88 

Epoch 68/1000


Batch in Progress:   0%|                                                     | 0/29 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.540849,    Accuracy: 71.04%,    Correct Counter: 650/915,    F1 Score: 0.75,    Precision: 0.65,    Recall:  0.88 

Validation Set:
    Average Loss:  0.590163,    Accuracy: 68.42%,    Correct Counter: 156/228,    F1 Score: 0.74,    Precision: 0.63,    Recall:  0.88 

Epoch 69/1000
Train Set:
    Average Loss:  0.540722,    Accuracy: 71.04%,    Correct Counter: 650/915,    F1 Score: 0.75,    Precision: 0.65,    Recall:  0.88 

Validation Set:
    Average Loss:  0.590360,    Accuracy: 68.42%,    Correct Counter: 156/228,    F1 Score: 0.74,    Precision: 0.63,    Recall:  0.88 

Epoch 70/1000


Batch in Progress: 100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 542.07it/s]

Train Set:
    Average Loss:  0.540603,    Accuracy: 71.04%,    Correct Counter: 650/915,    F1 Score: 0.75,    Precision: 0.65,    Recall:  0.88 

Validation Set:
    Average Loss:  0.590556,    Accuracy: 68.42%,    Correct Counter: 156/228,    F1 Score: 0.74,    Precision: 0.63,    Recall:  0.88 



In [27]:
torch.save(fusion_model.state_dict(), MODELS_PATH + "\KeepFeatures\FusionPath.pth")

# VotingFeatures (Late Fusion)

In [28]:
# Load image and Tabular model
image_model = get_densenet201(device=device, freeze=True, with_mlp=True, outputs=1)

tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

image_model.load_state_dict(torch.load(FM_PATH))
tabular_model.load_state_dict(torch.load(DM_PATH))

image_model, tabular_model = image_model.to(device).float(), tabular_model.to(device).float()

# load fusion model
tabular_path = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
)

image_path = get_densenet201(device=device, freeze=True, with_mlp=False, outputs=32)

fusion_path = nn.Sequential(
    nn.Linear(in_features=64, out_features=128),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

image_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\FundusPath.pth"))
tabular_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\DemographicPath.pth"))
fusion_path.load_state_dict(torch.load(MODELS_PATH + "\CombineFeatures\FusionPath.pth"))

tabular_path, image_path, fusion_path = tabular_path.to(device).float(), image_path.to(device).float(),\
                                                fusion_path.to(device).float()

fusion_model = dict()
fusion_model["image_path"], fusion_model["tabular_path"], fusion_model["fusion_path"] = image_path, \
                                                                                    tabular_path, fusion_path

In [29]:
models = {
    "image_model": image_model,
    "tabular_model": tabular_model,
    "fusion_model": fusion_model,
}

In [30]:
train_x, train_y = build_tabular_dataset(models, train_dataset, method="voting_features")
test_x, test_y = build_tabular_dataset(models, test_dataset, method="voting_features")

train_fusion_set = InputOutputDataset(train_x, train_y)
test_fusion_set = InputOutputDataset(test_x, test_y)

train_fusion_loader = DataLoader(train_fusion_set, batch_size=32)
test_fusion_loader = DataLoader(test_fusion_set, batch_size=32)

## Classifier using all scores

### XGBOOST

In [31]:
boost = xgb.XGBClassifier(tree_method='gpu_hist', objective="binary:logistic")

In [32]:
param_grid = {'max_depth': [3,6,10],
           'learning_rate': [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.3],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

clf = GridSearchCV(estimator=boost, 
                   param_grid=param_grid,
                   scoring='f1', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed: 16.2min finished


0.7591363605873314


In [33]:
with open(MODELS_PATH+'/VotingFeatures/XGBParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

### SVM

In [34]:
svm = SVC()

In [35]:
param_grid = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']},
    {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'degree': [1,2,3], 'kernel': ['poly']}
]

clf = GridSearchCV(estimator=svm, 
                   param_grid=param_grid,
                   scoring='f1', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

Fitting 5 folds for each of 68 candidates, totalling 340 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.7643395362141236


[Parallel(n_jobs=1)]: Done 340 out of 340 | elapsed:  3.4min finished


In [36]:
with open(MODELS_PATH+ '/VotingFeatures/SVMParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

### FCNN

In [37]:
fusion_model = nn.Sequential(
    nn.Linear(in_features=3, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=32),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=32, out_features=16),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=16, out_features=8),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=8, out_features=1),
)

fusion_model = fusion_model.to(device).float()

In [38]:
model = {
    "image_model": None,
    "tabular_model": None,
    "fusion_model": fusion_model,
}

early_stop = {
    "patience": 20,
    "min_delta": 0,
    "multip": 10
}

optimizer = torch.optim.Adam([
    {'params': fusion_model.parameters(), 'lr': 1e-4}],
)

In [39]:
epochs = 1000
metrics = train_val(epochs=epochs, models=model, criterion=criterion, optimizer=optimizer, train_loader=train_fusion_loader,
                    val_loader=test_fusion_loader, device=device, early_stop=early_stop)

Batch in Progress:  66%|████████████████████████████▏              | 19/29 [00:00<00:00, 175.53it/s]

Beginning Training: 

Epoch 1/1000


Batch in Progress:   0%|                                                      | 0/8 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.701557,    Accuracy: 51.48%,    Correct Counter: 471/915,    F1 Score: 0.00,    Precision: 1.00,    Recall:  0.00 



Batch in Progress: 100%|███████████████████████████████████████████| 29/29 [00:00<00:00, 194.91it/s]

...

Train Set:
    Average Loss:  0.535935,    Accuracy: 71.69%,    Correct Counter: 656/915,    F1 Score: 0.76,    Precision: 0.65,    Recall:  0.92 

Validation Set:
    Average Loss:  0.621714,    Accuracy: 66.23%,    Correct Counter: 151/228,    F1 Score: 0.72,    Precision: 0.61,    Recall:  0.89 

Epoch 67/1000
Train Set:
    Average Loss:  0.535806,    Accuracy: 71.69%,    Correct Counter: 656/915,    F1 Score: 0.76,    Precision: 0.65,    Recall:  0.92 

Validation Set:
    Average Loss:  0.621768,    Accuracy: 66.23%,    Correct Counter: 151/228,    F1 Score: 0.72,    Precision: 0.61,    Recall:  0.89 

Epoch 68/1000


Batch in Progress:   0%|                                                     | 0/29 [00:00<?, ?it/s]

Train Set:
    Average Loss:  0.535666,    Accuracy: 71.69%,    Correct Counter: 656/915,    F1 Score: 0.76,    Precision: 0.65,    Recall:  0.92 

Validation Set:
    Average Loss:  0.621752,    Accuracy: 66.23%,    Correct Counter: 151/228,    F1 Score: 0.72,    Precision: 0.61,    Recall:  0.89 

Epoch 69/1000
Train Set:
    Average Loss:  0.535539,    Accuracy: 71.69%,    Correct Counter: 656/915,    F1 Score: 0.76,    Precision: 0.65,    Recall:  0.92 

Validation Set:
    Average Loss:  0.621738,    Accuracy: 66.23%,    Correct Counter: 151/228,    F1 Score: 0.72,    Precision: 0.61,    Recall:  0.89 

Epoch 70/1000


Batch in Progress: 100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 685.55it/s]

Train Set:
    Average Loss:  0.535424,    Accuracy: 71.69%,    Correct Counter: 656/915,    F1 Score: 0.76,    Precision: 0.65,    Recall:  0.92 

Validation Set:
    Average Loss:  0.621745,    Accuracy: 66.23%,    Correct Counter: 151/228,    F1 Score: 0.72,    Precision: 0.61,    Recall:  0.89 



In [40]:
torch.save(fusion_model.state_dict(), MODELS_PATH + "\VotingFeatures\FusionPath.pth")